# Azure AI Agents s podrškom za Model Context Protocol (MCP)

Ovaj bilježnik pokazuje kako koristiti Azure AI Agents s alatima Model Context Protocol (MCP) za stvaranje inteligentnog agenta koji može koristiti vanjske MCP poslužitelje za poboljšane mogućnosti.


## Instalirajte potrebne NuGet pakete

Prvo, trebamo instalirati paket Azure AI Agents Persistent koji pruža osnovnu funkcionalnost za rad s Azure AI agentima.


## Prednosti autentifikacije bez ključeva

Ovaj bilježnik demonstrira **autentifikaciju bez ključeva** koja pruža nekoliko prednosti:
- ✅ **Nema upravljanja API ključevima** - Koristi autentifikaciju temeljenu na Azure identitetu
- ✅ **Poboljšana sigurnost** - Nema pohranjenih tajni u kodu ili konfiguraciji
- ✅ **Automatska rotacija vjerodajnica** - Azure upravlja životnim ciklusom vjerodajnica
- ✅ **Pristup temeljen na ulogama** - Koristi Azure RBAC za precizno definirane dozvole

`DefaultAzureCredential` automatski će koristiti najbolji dostupni izvor vjerodajnica:
1. Upravljani identitet (kada se pokreće u Azureu)
2. Azure CLI vjerodajnice (tijekom razvoja)
3. Vjerodajnice iz Visual Studija
4. Varijable okruženja (ako su konfigurirane)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Instalirajte paket Azure Identity za autentifikaciju s Azure uslugama koristeći DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Uvoz potrebnih imenskih prostora

Uvezite potrebne imenske prostore za Azure AI Agents i Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfiguriranje Azure AI Agent Klijenta (Autentifikacija bez ključa)

Postavite varijable konfiguracije i kreirajte PersistentAgentsClient koristeći **autentifikaciju bez ključa**:
- **projectEndpoint**: Krajnja točka Azure AI Foundry projekta
- **modelDeploymentName**: Naziv implementiranog AI modela (GPT-4.1 nano)
- **mcpServerUrl**: URL MCP poslužitelja (Microsoft Learn API)
- **mcpServerLabel**: Oznaka za identifikaciju MCP poslužitelja
- **DefaultAzureCredential**: Koristi upravljani identitet, Azure CLI ili druge izvore vjerodajnica (API ključevi nisu potrebni)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Kreirajte MCP definiciju alata

Kreirajte MCP definiciju alata koja se povezuje s Microsoft Learn MCP poslužiteljem. Ovo će omogućiti agentu pristup sadržaju i dokumentaciji na Microsoft Learn platformi.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Kreirajte AI agenta

Kreirajte trajnog AI agenta s određenim modelom i MCP alatima. Agent je konfiguriran s:
- GPT-4.1 nano modelom
- Uputama za korištenje MCP alata za pomoć
- Pristupom Microsoft Learn MCP serveru


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Kreiraj nit i pošalji poruku

Kreirajte razgovornu nit i pošaljite korisničku poruku s pitanjem o razlici između Azure OpenAI i OpenAI. Ovo će testirati sposobnost agenta da koristi MCP alate za pružanje točnih informacija.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfiguriranje MCP alata resursa (bez ključa)

Postavite resurse MCP alata. Za potpuno pristup bez ključa, možete ukloniti prilagođene zaglavlja ako MCP poslužitelj podržava autentifikaciju temeljenu na Azure identitetu. Primjer u nastavku pokazuje kako dodati zaglavlja ako je potrebno, ali za scenarije bez ključa, ona možda neće biti potrebna.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Pokreni Agent Run

Kreirajte i pokrenite proces za obradu korisnikove poruke. Agent će koristiti konfigurirane MCP alate i resurse za generiranje odgovora.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Praćenje izvođenja i upravljanje odobrenjima alata (bez ključa)

Pratite status izvođenja agenta i upravljajte potrebnim odobrenjima alata. Ova petlja:
1. Čeka da se izvođenje završi ili zahtijeva akciju
2. Automatski odobrava pozive MCP alata kada je to potrebno
3. Za autentifikaciju bez ključa, zaglavlja možda neće biti potrebna ako MCP poslužitelj podržava Azure identitet


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Prikaz rezultata razgovora

Dohvatite i prikažite sve poruke u niti, prikazujući i korisnikovo pitanje i odgovor agenta. Poruke se prikazuju kronološkim redoslijedom s vremenskim oznakama i oznakama uloga.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
